## **Modelagem**

Nesse notebook constam todos os trabalhos de preparação de dados e modelagem com o objetivo de desenvolver o modelo final, com o mais performático.

In [3]:
from pandas import read_csv
import os

In [4]:
data_dir = 'data'
file_name = 'relatos_ml_gzip.csv'

dir_path = os.path.dirname(os.getcwd())
data_path = os.path.join(dir_path, data_dir)
file_path = os.path.join(data_path, file_name)

In [5]:
data = read_csv(file_path, sep='|', encoding='utf-8', compression='gzip')
data.head()

,company_name,status,report,company_response,uf,respondido,dias_para_resposta,nota
0,Natura,0,"A Natura vem me fazendo ligações, me passando ...","Olá Larissa, boa noite!\n\nTudo bem? Esperamos...",SP,1,0,1
1,Enel Distribuição Rio (Ampla),1,PARCELAMENTO MUTIRAO ENEL MARICÁ,"Olá, Luiz!\nAgradecemos o seu contato e a opor...",RJ,1,0,3
2,Dog Life,1,Boa tarde!\n\nFiz a contratação do plano Dog L...,"Olá, Janaina,\n\nAgradecemos seu contato e lam...",AM,1,0,5
3,Serasa Experian,0,Foi realizado a consulta do meu CPF pela empre...,"Oi, ROSANE. Tudo bem?\n\n\nAcabamos de respond...",RJ,1,0,1
4,Serasa Experian,1,Em 03 de março de 2025 verifiquei consultei a ...,"Oi, Nelson. Tudo bem?\n\n\nAcabamos de respond...",SP,1,0,3


### **Tratamento Textual:**

1. Remoção de **acentos e caracteres especiais**  
2. Remoção de **stopwords**  
3. **Tokenização**  
4. **Lematização**  
5. **Vetoriza** o texto usando `TfidfVectorizer`

In [6]:
# Import modules
import re
import unicodedata
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [7]:
# Downloading resources
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('portuguese'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


---

### **Term Frequency – Inverse Document Frequency**.

É uma técnica que mede **a importância de uma palavra** em um documento, considerando **todo o conjunto de documentos** (ou corpus). Serve para destacar palavras **relevantes**, ignorando palavras muito comuns (como "de", "a", "o").

### **Cálculo:**

#### 1. **TF (Term Frequency)** – Frequência da palavra no documento

$$
TF(t, d) = \frac{\text{Número de vezes que o termo } t \text{ aparece no documento } d}{\text{Total de termos no documento } d}
$$

#### 2. **IDF (Inverse Document Frequency)** – Importância da palavra no corpus

$$
IDF(t) = \log\left( \frac{N}{1 + n_t} \right)
$$

- \( N \): total de documentos
- \( n_t \): número de documentos em que o termo \( t \) aparece

Quanto **mais documentos** contêm a palavra, **menor** o IDF (menos importante ela é).

#### 3. **TF-IDF final**:

$$
TF\text{-}IDF(t, d) = TF(t, d) \times IDF(t)
$$

#### **Conclusão**:
- Palavras **frequentes num documento**, mas **raras no corpus**, recebem **pesos altos**.
- Palavras **comuns em todo lugar** (ex: "é", "o", "a") acabam com peso **baixo ou zero**.

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextCleaner(BaseEstimator, TransformerMixin):
    """Custom transformer to clean and preprocess text data."""

    def __init__(self, spacy_model:str = 'pt_core_news_sm', remove_accents=False):
        import spacy as spacy_lib
        self.spacy_lib = spacy_lib
        self.spacy_model = spacy_model
        self.remove_accents = remove_accents
        self.spacy = None  # spaCy será carregado no fit

    def fit(self, X, y=None):
        """Carrega o modelo spaCy quando fit é chamado."""
        self.spacy = self.spacy_lib.load('pt_core_news_sm')
        return self

    def clean_text(self, text):
        """Remove caracteres especiais e, opcionalmente, acentos."""
        text = text.lower()
        text = text.replace('\n', ' ')
        text = re.sub(r'[^a-záàâãéèêíïóôõöúçñü\s]', '', text, flags=re.IGNORECASE)
        if self.remove_accents:
            text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
            text = re.sub(r'[^a-z\s]', '', text)
        return text

    def preprocess_text(self, text):
        """Pré-processa o texto: limpeza e lematização."""
        clean_text = self.clean_text(text)
        doc = self.spacy(clean_text)
        tokens = [token.lemma_ for token in doc if token.text not in stop_words and not token.is_punct and not token.is_space]
        return ' '.join(tokens)

    def transform(self, X):
        """Transforma os dados de entrada, aplicando o pré-processamento."""
        return [self.preprocess_text(text) for text in X]

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Pipeline de pré-processamento e vetorização
pipeline_report = Pipeline([
    ('preprocessador', TextCleaner(spacy_model='pt_core_news_sm', remove_accents=False)),
    ('vetorizador', TfidfVectorizer(max_features=5000, max_df=0.9, min_df=5))
])

# Pipeline para 'company_response'
pipeline_response = Pipeline([
    ('preprocessador', TextCleaner(spacy_model='pt_core_news_sm', remove_accents=False)),
    ('vetorizador', TfidfVectorizer(max_features=5000, max_df=0.9, min_df=5))
])

In [10]:
data['vectorized_report'] = list(pipeline_report.fit_transform(data['report']).toarray())
data['vectorized_company_response'] = list(pipeline_response.fit_transform(data['company_response']).toarray())

In [165]:
data['vectorized_report'] = data['vectorized_report'].to_numpy()
data['vectorized_company_response'] = data['vectorized_company_response'].to_numpy()

In [12]:
import numpy as np

class Indexer:

    def __init__(self, data):
        self.data = data
        self.index = 0
        self.index_map = {}
        self.index_map_inv = {}
    
    def fit(self, column):
        unique_values = column.unique()
        for value in unique_values:
            if value not in self.index_map:
                self.index_map[value] = self.index
                self.index_map_inv[self.index] = value
                self.index += 1
        return self

    def transform(self, text):
        """Return a sparse matrix with binary values if the column is categorical."""
        if text in self.index_map:
            index = self.index_map[text]
            sparse = [0] * len(self.index_map)
            sparse[index] = 1
        else:
            sparse = [0] * len(self.index_map)
        
        return np.array(sparse)
    
    def inverse_transform(self, one_hot_vector):
        """Retorna o valor original com base no vetor one-hot."""
        if not isinstance(one_hot_vector, list):
            raise ValueError("Esperado um vetor one-hot como lista.")

        try:
            index = one_hot_vector.index(1)
            return self.index_map_inv.get(index, None)
        except ValueError:
            return None  # Nenhum valor 1 no vetor

In [32]:
indexer_uf = Indexer(data['uf']).fit(data['uf'])
indexer_comp = Indexer(data['company_name']).fit(data['company_name'])

data['uf_bin'] = data['uf'].apply(lambda x: indexer_uf.transform(x))
data['company_name_bin'] = data['company_name'].apply(lambda x: indexer_comp.transform(x))

### **Separação Treino x Teste**

Aqui, com um ensemble, junto as informações geradas a partir do tratamento e crio um vetor único, técnica chamada _ensemble_.

In [80]:
# Matriz de entrada X
X = np.stack([
    np.concatenate([
        row['vectorized_report'],
        row['vectorized_company_response'],
        #row['uf_bin'],
        #row['company_name_bin'],
        np.array([row['respondido']]),           
        np.array([row['dias_para_resposta']])    
    ])
    for _, row in data.iterrows()
])

# Target
y = data['status'].values

- **Treino: 80%**
- **Teste: 20%**

In [81]:
from sklearn.model_selection import train_test_split

# Separação em treino e teste (com estratificação pela variável alvo)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

### ``Regressão Logística``

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Inicializa o modelo
logreg = LogisticRegression(max_iter=1000, random_state=42)

# Treina o modelo
logreg.fit(X_train, y_train)

# Faz previsões
y_pred = logreg.predict(X_test)

In [42]:
# Avaliação do modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7151851851851851

Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.70      0.70      1268
           1       0.73      0.73      0.73      1432

    accuracy                           0.72      2700
   macro avg       0.71      0.71      0.71      2700
weighted avg       0.72      0.72      0.72      2700



Claro, Pedro! Aqui vai um resumo bem direto dos principais **métricas de avaliação** em classificação 👇

---

#### **Acurácia** (Accuracy)
É a proporção de acertos **no total de previsões**.

$$
\text{Acurácia} = \frac{\text{Nº de previsões corretas}}{\text{Total de previsões}}
$$

> Boa quando as classes estão balanceadas.

#### **Precisão** (Precision)
Entre os que o modelo **disse que eram positivos**, quantos realmente são?

$$
\text{Precisão} = \frac{\text{Verdadeiros Positivos}}{\text{Verdadeiros Positivos + Falsos Positivos}}
$$

> Útil quando **falsos positivos são críticos** (ex: diagnosticar alguém saudável como doente).

#### **Recall** (Sensibilidade)
Entre os que **realmente eram positivos**, quantos o modelo encontrou?

$$
\text{Recall} = \frac{\text{Verdadeiros Positivos}}{\text{Verdadeiros Positivos + Falsos Negativos}}
$$

> Útil quando **falsos negativos são mais graves** (ex: deixar de diagnosticar uma doença).

#### **F1-Score**
É a **média harmônica** entre precisão e recall. Balanceia os dois.

$$
F1 = 2 \cdot \frac{\text{Precisão} \cdot \text{Recall}}{\text{Precisão + Recall}}
$$

> Bom quando há **desequilíbrio entre classes** e você quer um equilíbrio entre **erros tipo I e II**.

##### **GridSearch**

In [82]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modelo base (Regressão Logística)
logreg_tuned = LogisticRegression(max_iter=1000, random_state=42)

param_grid = [
    {
        'penalty': ['l1'],
        'C': [0.01, 0.1, 1, 10, 100],
        'solver': ['liblinear', 'saga']
    },
    {
        'penalty': ['l2'],
        'C': [0.01, 0.1, 1, 10, 100],
        'solver': ['lbfgs', 'liblinear', 'saga']
    },
    {
        'penalty': ['elasticnet'],
        'C': [0.01, 0.1, 1, 10, 100],
        'solver': ['saga'],
        'l1_ratio': [0.5]
    },
    {
        'penalty': ['none'],
        'solver': ['lbfgs', 'saga']
    }
]


# RandomSearch
random_search = RandomizedSearchCV(
    logreg_tuned,
    param_distributions=param_grid,  
    n_iter=20, 
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [83]:
# Executa a busca
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


KeyboardInterrupt: 

In [ ]:
# Melhor modelo
best_model = grid_search.best_estimator_
print("Melhores parâmetros encontrados:", grid_search.best_params_)

In [ ]:
# Avaliação
y_pred = best_model.predict(X_test)
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))

---

### `Naive Bayes`

In [79]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Inicializa o modelo Naive Bayes
nb = MultinomialNB()

# Treina o modelo
nb.fit(X_train, y_train)

# Faz previsões
y_pred = nb.predict(X_test)

# Avaliação
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

Acurácia: 0.7022222222222222

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.66      0.74      0.70      1268
           1       0.75      0.66      0.70      1432

    accuracy                           0.70      2700
   macro avg       0.70      0.70      0.70      2700
weighted avg       0.71      0.70      0.70      2700



---

### `RandomForestClassifier`
- **O que é:** Um **ensemble** de muitas árvores de decisão treinadas de forma independente.
- **Como funciona:** Cada árvore aprende com um subconjunto aleatório dos dados e dos recursos (features), e a predição final é feita por **votação da maioria**.
- **Vantagem:** Robusto contra overfitting e funciona bem com dados variados.

---

### `GradientBoostingClassifier`
- **O que é:** Também é um ensemble de árvores, mas construído de forma **sequencial**.
- **Como funciona:** Cada nova árvore tenta **corrigir os erros** da anterior, usando o gradiente do erro como orientação.
- **Vantagem:** Mais preciso que Random Forest em muitos casos, mas pode ser mais sensível a overfitting se não for bem ajustado.

---

### `XGBClassifier` 
- **O que é:** Uma implementação otimizada e extremamente eficiente de Gradient Boosting.
- **Como funciona:** Mesma lógica do Gradient Boosting, mas com **melhor desempenho** (usa regularização, paralelismo, e outras otimizações).
- **Vantagem:** Muito usado em competições de machine learning (ex: Kaggle), por ser rápido e entregar alta performance.

In [43]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Gradient Boosting
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gb.fit(X_train, y_train)

# XGBoost
xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

### `Rede Neural`

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

In [46]:
# Padronização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [56]:
# Criando o modelo
from tensorflow.keras import regularizers
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.4),
    Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [57]:
# Compilar
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [59]:
# Treinar
history = model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1, callbacks=[early_stop])

Epoch 1/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.5611 - loss: 1.1452 - val_accuracy: 0.6787 - val_loss: 0.6468
Epoch 2/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6936 - loss: 0.6871 - val_accuracy: 0.6792 - val_loss: 0.6136
Epoch 3/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7609 - loss: 0.5548 - val_accuracy: 0.6972 - val_loss: 0.6018
Epoch 4/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8234 - loss: 0.4456 - val_accuracy: 0.6903 - val_loss: 0.6251
Epoch 5/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8498 - loss: 0.3756 - val_accuracy: 0.6907 - val_loss: 0.6456
Epoch 6/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8905 - loss: 0.3134 - val_accuracy: 0.6806 - val_loss: 0.7257
Epoch 7/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8996 - loss: 0.2735 - val_accuracy: 0.6866 - val_loss: 0.8214
Epoch 8/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9264 - loss: 0.2310 - val_accu

In [61]:
# Avaliação
y_pred_prob = model.predict(X_test_scaled).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

y_test_prob = model.predict(X_train_scaled).ravel()
y_test_pred = (y_test_prob > 0.5).astype(int)

print(classification_report(y_test, y_pred))

85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
              precision    recall  f1-score   support

           0       0.67      0.64      0.66      1268
           1       0.70      0.72      0.71      1432

    accuracy                           0.68      2700
   macro avg       0.68      0.68      0.68      2700
weighted avg       0.68      0.68      0.68      2700



In [ ]:
print(classification_report(y_test, y_pred))